In [62]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.compose import ColumnTransformer

from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier, GradientBoostingRegressor

import numpy as np
import pandas as pd

from scipy import sparse

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [12]:
#IMPORT MODULES
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder, FunctionTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.cluster import KMeans

/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/kpandey/anaconda3/envs/nlp_course/lib/python3.7/site-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [5]:
#GET THE DATAFRAME
df = pd.read_csv('amazon_reviews.tsv', sep = '\t')

In [6]:
df.columns

Index(['eDOC_ID', 'LABEL', 'RATING', 'VERIFIED_PURCHASE', 'PRODUCT_CATEGORY',
       'PRODUCT_ID', 'PRODUCT_TITLE', 'REVIEW_TITLE', 'REVIEW_TEXT',
       'POS_COUNTS', 'NUM NOUNS', 'NUM VERBS', 'NUM ADJECTIVES', 'NUM ADVERBS',
       'REVIEW_LENGTH', 'SENTIMENT SCORE'],
      dtype='object')

In [7]:
#RENAME COLUMNS
df.rename(columns={'NUM NOUNS':'NUM_NOUNS'}, inplace=True)
df.rename(columns={'NUM VERBS':'NUM_VERBS'}, inplace=True)
df.rename(columns={'NUM ADJECTIVES':'NUM_ADJECTIVES'}, inplace=True)
df.rename(columns={'NUM ADVERBS':'NUM_ADVERBS'}, inplace=True)
df.rename(columns={'SENTIMENT SCORE':'SENTIMENT_SCORE'}, inplace=True)

In [20]:
#ADD A TITLE LENGTH COLUMN
df['TITLE_LENGTH'] = df['REVIEW_TITLE'].apply(lambda d: len(d))

#ADD AN AVERAGE-RATING COLUMN
average_ratings = df.groupby('PRODUCT_ID')['RATING'].mean()

df['AVERAGE_RATING'] = df['PRODUCT_ID'].map(average_ratings)

#ADD A RATING DEVIATION COLUMN
df['RATING_DEVIATION'] = df['RATING']-df['AVERAGE_RATING']


#ADD A NUMBER OF REVIEWS COLUMN
num_reviews = df.groupby('PRODUCT_ID').size()

df['NUM_REVIEWS'] = df['PRODUCT_ID'].map(num_reviews)

In [85]:
#SAVE AS CSV FILE
df.to_csv('amazon_reviews3.tsv', sep='\t')

In [44]:
df['VERIFIED_PURCHASE'].value_counts()
len(df['REVIEW_TEXT'].tolist())

21000

In [54]:
#X -> FEATURE
features = ['RATING', 'VERIFIED_PURCHASE', 'NUM_NOUNS', 'NUM_VERBS', 
            'NUM_ADJECTIVES', 'NUM_ADVERBS','SENTIMENT_SCORE', 'TITLE_LENGTH', 'REVIEW_LENGTH','RATING_DEVIATION']

text_columns = ['REVIEW_TEXT', 'REVIEW_TITLE']
numeric_columns = ['RATING','VERIFIED_PURCHASE', 'SENTIMENT_SCORE',
                  'REVIEW_LENGTH', 'TITLE_LENGTH', 'NUM_NOUNS',
                  'NUM_ADJECTIVES','NUM_ADVERBS','NUM_VERBS',
                   'RATING_DEVIATION', 'VERIFIED_PURCHASE'
                  ]

X = df[features]
Y = df['LABEL']

In [55]:
#TRAIN-TEST SPLIT
X_TRAIN, X_TEST, Y_TRAIN, Y_TEST = train_test_split(X, Y, test_size = 0.2, random_state = 42)

In [69]:
# Fit and transform the training data
# Create the KNN classifier
knn = K(n_estimators=50)

knn.fit(X_TRAIN, Y_TRAIN)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [70]:
predictions = knn.predict(X_TEST)

In [71]:
print(classification_report(Y_TEST, predictions))

              precision    recall  f1-score   support

        Fake       0.82      0.74      0.77      2115
    Not Fake       0.76      0.83      0.79      2085

   micro avg       0.78      0.78      0.78      4200
   macro avg       0.79      0.78      0.78      4200
weighted avg       0.79      0.78      0.78      4200



In [72]:
#HYPERPARAMETER TUNING TO FIND MOST EFFICIENT ARGUMENT FOR KNN
from sklearn.model_selection import GridSearchCV

In [73]:
#RANGE TO SEARCH FOR VALUE OF K
param_grid = {'n_neighbors': range(1,25)}

In [74]:
#KNN CLASSIFIER
knn_classifier = KNeighborsClassifier()

In [75]:
#GRID SEARCH OVER THE PARAMETER RANGE [cross-validation]
grid_search = GridSearchCV(knn_classifier, param_grid, cv=5)

In [76]:
grid_search.fit(X_TRAIN, Y_TRAIN)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'n_neighbors': range(1, 25)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=0)

In [78]:
#BEST NUMBER OF NEIGHBORS
best_n_neighbors = grid_search.best_params_['n_neighbors']

In [79]:
#NEW KNN CLASSIFIER
best_knn_classifier = KNeighborsClassifier(n_neighbors = best_n_neighbors)

In [80]:
#FIT ON TRAINING DATA
best_knn_classifier.fit(X_TRAIN, Y_TRAIN)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=17, p=2,
           weights='uniform')

In [81]:
#MAKE PREDICTIONS
predictions = best_knn_classifier.predict(X_TEST)

In [82]:
#CLASSIFICATION REPORT
print(classification_report(Y_TEST, predictions))
#BAD PRECISION OF JUST 61%

              precision    recall  f1-score   support

        Fake       0.61      0.59      0.60      2115
    Not Fake       0.60      0.62      0.61      2085

   micro avg       0.60      0.60      0.60      4200
   macro avg       0.60      0.60      0.60      4200
weighted avg       0.60      0.60      0.60      4200

